In [1]:
import flwr as fl
from config import NUM_CLIENTS
from model import Net
from main import simulate_cifar


## Fed avg with parameter initialization

In [ ]:
params = Net().get_parameters()

# Pass parameters to the Strategy for server-side parameter initialization
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(params),
)

simulate_cifar(strategy)

57.5%